                                                     A Private Stable Matching Algorithm
                                                              Philippe Golle



A stable matching algorithm takes as inputs the preferences of suitors for reviewers and of reviewers for suitors, and outputs a stable matching between them. A match is a bijection between suitors and reviewers, or equivalently a set of
n marriages between the n suitors and the n reviewers. A stable match is a match such that there is no
suitor and reviewer that both like each other better than their respective partners. Existing stable matching algorithms reveal the preferences of all participants, as well as the history of matches made and broken in the course of computing a stable match. This information leakage not only violates the privacy of participants, but also leaves matching algorithms vulnerable to manipulation. To address these limitations, this paper proposes a private stable matching algorithm, based on the famous algorithm of Gale and Shapley.


In [ ]:
##### SETUP ############
import sys
import numpy as np
import random
import gmpy2
from phe import paillier
public_key, private_key = paillier.generate_paillier_keypair()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

The private algorithm is run by a number of independent parties called the Matching Authorities. As long as a majority of Matching Authorities are honest, the protocol correctly outputs a stable match, and reveals no other information than what can be learned from that match and from the preferences of participants controlled by the adversary. The security and privacy of our protocol are based on re-encryption mix networks and on an additively homomorphic semantically secure public-key encryption scheme such as Paillier. The implementation so far assumes a single honest matching authority/server to test the algorithm, and can be extended to multiple matching authorities using secure multiparty computation in the future.


                                                    CRYPTOGRAPHIC BUILDING BLOCKS

In [ ]:
'''
Oblivious test of plaintext equality.

An oblivious test of plaintext equality lets the joint holders of the decryption key determine whether m1 = m2 without revealing any other information, i.e, 
Test whether two ciphertexts have same origin plaintext
'''
def EQ_TEST(x, y, private_key):
    a = x - y
    if private_key.decrypt(a) == 0:
        return 1
    else:
        return 0

In [ ]:
'''
Repeated test of plaintext equality. 

The protocol INDEX(a, E(ρ)) takes as input a vector a = (E(a1), . . . , E(an)) of n Paillier ciphertexts and an additional Paillier ciphertext   E(ρ) such that there exists one and only one value i ∈ {1, . . . , n} for which ρ = ai.
The protocol outputs the index i such that ai = ρ. The protocol INDEX can be implemented with n instances of EQTEST.
'''
def INDEX(a, ciphertext, private_key):
    index = 0
    while True:
        if EQ_TEST(a[index], ciphertext, private_key) == 1:
            break
        index += 1
    return index

In [ ]:
'''
Finding the larger of 2 plaintexts

Let E(m1) and E(m2) be two Paillier ciphertexts such that m1, m2 ∈ {0, . . . , n − 1} and m1 is not equal to m2. COMPARE(E(m1), E(m2)) outputs true if m1 > m2 and false otherwise, without leaking any other information. Note that m1 > m2 if and only if one of the ciphertexts Di is an encryption of 0.
The protocol proceed as follows:

'''
def COMPARE(c1, c2, private_key, n):
    D = []
    '''
    For i = 1, . . . , n − 1, the matching authorities compute ciphertext
    Di = E(m1−m2−i) using Paillier’s additive homomorphism
    '''
    for i in range(1, n):
        D_i = c1 - c2 - public_key.encrypt(i)
        D.append(D_i)
    '''
    The matching authorities mix (i.e. re-encrypt and permute) the set of ciphertexts D1, . . . , Dn−1
    '''
    D = [re_encrypt(i) for i in D]
    sigma = np.random.permutation(len(D))
    D = [D[i] for i in sigma]

    '''
    The matching authorities then compute EQTEST(Di, E(0)) for i = 1, . . . , n−1. If an equality is found, they output true, otherwise they         output false.
    '''
    for i in range(len(D)):
        if EQ_TEST(D[i], public_key.encrypt(0), private_key) == 1:
            return True

    return False

HELPER FUNCTIONS

In [ ]:
def re_encrypt(c1):
    temp = public_key.encrypt(0)
    temp = c1 + temp
    return temp

In [ ]:
def add_1(c1):
    temp = public_key.encrypt(1)
    temp = c1 + temp
    return temp

In [ ]:
def pi_per(ciphers):
    ciphers = np.array([re_encrypt(c) for c in ciphers])
    ciphers = np.array(ciphers[pi].tolist())
    return ciphers

INPUT SUBMISSION

In [ ]:
'''
For the purpose of implementation, generating preferences at random. In actuality to be submitted as encrypted preferences by clients.
'''
def gen_prefs(n):
    l = np.empty([n, n]).astype(int).tolist()
    for i in range(n):
        pi = np.random.permutation(n).astype(int).tolist()
        l[i] = [x for x in pi]
    return l

In [ ]:
'''
Encrypting randomly generated preferences
'''
def gen_enc(n, preferences):
    men = dict()
    for i in range(n):
        men[i] = [public_key.encrypt(x) for x in preferences[i]]
    return men

BID CREATION

In [ ]:
'''
bids created as per the exact format in the paper
A bid by itself, as defined below, is called a free bid because it is not paired up with a woman.
A bid paired up with a woman is called an engaged bid
'''
def create_bids():
    a = list(range(1, n+1))

    for i in range(2*n):
        women_pref = []
        for j in range(n):
            women_pref.append(women[j][i])

        bids[i] = np.array([public_key.encrypt(i+1), np.array(men[i]), np.array([public_key.encrypt(x)
                                                                                 for x in a]), np.array(women_pref), public_key.encrypt(0)])  

                                                            MIXING
The Paillier cryptosystem allows for semantically secure re-encryption of ciphertexts. Since bids (both free and engaged) are made up of Paillier ciphertexts, they can be re-encrypted, and in particular they can be mixed with a re-encryption mix network. We consider two types of mixing for bids: “external” mixing and “internal” mixing. 

EXTERNAL MIXING: 
External mixing takes as input a set of bids, either all
free or all engaged, and mixes them in a way that hides the order of the bids
but preserves the internal position of ciphertexts within a bid. Let us consider an initial ordering of k free bids W1, . . . , Wk and let σ
be a permutation on k elements. The external mixing operation re-encrypts all
the Paillier ciphertexts in all the bids (preserving the order of ciphertexts within
each bid) and outputs Wσ(1), . . . , Wσ(k).

In [ ]:
def External_Mix_Free(bids):
    if len(bids) == 0:
        return bids

    for i in range(len(bids)):
        bids[i] = np.array([re_encrypt(bids[i][0]), np.array(np.apply_along_axis(lambda x: re_encrypt(x), 0, bids[i][1])), np.array(
        np.apply_along_axis(lambda x: re_encrypt(x), 0, bids[i][2])), np.array(np.apply_along_axis(lambda x: re_encrypt(x), 0, bids[i][3])),             re_encrypt(bids[i][4])])

    sigma = np.random.permutation(len(bids))
    bids = bids[sigma]
    return bids

In [ ]:
def External_Mix_Engaged(bids):
    if len(bids) == 0:
        return bids

    for i in range(len(bids)):
        bids[i] = np.array([np.array([re_encrypt(bids[i][0][0]), np.array(np.apply_along_axis(lambda x: re_encrypt(x), 0, bids[i][0][1])),               np.array(np.apply_along_axis(lambda x: re_encrypt(x), 0, bids[i][0][2])), np.array(np.apply_along_axis(lambda x: re_encrypt(x), 0, bids          [i][0][3])), re_encrypt(bids[i][0][4])]), re_encrypt(bids[i][1]), re_encrypt(bids[i][2])])

    sigma = np.random.permutation(len(bids))
    bids = bids[sigma]
    return bids

INTERNAL MIXING: . Internal mixing takes as input a set of bids that may
contain both free and engaged bids. These bids are mixed “internally” in a way
that hides the order of a subset of the ciphertexts within the bids but preserves
the order of the bids themselves. More precisely, let us consider a set of k bids
and let π be a permutation on n elements. The bids in the set are processed one
by one, and output in the same order as they were given as input. 


In [ ]:
def Internal_Mix_Free(bids):
    if len(bids) == 0:
        return bids

    for i in range(len(bids)):
        bids[i] = np.array([bids[i][0], pi_per(bids[i][1]), pi_per(
            bids[i][2]), pi_per(bids[i][3]), bids[i][4]])

    return bids

In [ ]:
def Internal_Mix_Engaged(bids):
    if len(bids) == 0:
        return bids

    for i in range(len(bids)):
        bids[i] = np.array([np.array([bids[i][0][0], pi_per(bids[i][0][1]), pi_per(
            bids[i][0][2]), pi_per(bids[i][0][3]), bids[i][0][4]]), bids[i][1], bids[i][2]])
    return bids

SETUP

In [ ]:
##### SETUP ############
men_preferences = gen_prefs(n)
women_preferences = gen_prefs(n)
men = gen_enc(n, men_preferences)
women = gen_enc(n, women_preferences)

# ##### INPUT SUBMISSION ############

# #### ADDITION OF FAKE MEN ########
# # The matching authorities define an additional n fake men
for i in range(n, 2*n):
    pi = np.random.permutation(n).tolist()
    men[i] = [public_key.encrypt(x) for x in pi]


# # augmented preferences for woman is (i-1) for i in (n+1,2n)
augment = list(range(n, 2*n))
augment_ranklist = [public_key.encrypt(x) for x in augment]
for i in range(0, n):
    women[i].extend(augment_ranklist)

In [ ]:
# ######## BID CREATION #########
bids = dict()
create_bids()
Free_bids, Engaged_bids = [], []


for i in range(n):
    Free_bids.append(bids[i])
    Engaged_bids.append(
        np.array([bids[i+n], public_key.encrypt(i+1), women[i][i+n]]))

Free_bids = np.array(Free_bids)
Engaged_bids = np.array(Engaged_bids)

# global permutation pi 
pi = np.random.permutation(n)



###### Initial Mixing #########
Free_bids = External_Mix_Free(Free_bids)
Engaged_bids = External_Mix_Engaged(Engaged_bids)
Free_bids = Internal_Mix_Free(Free_bids)
Engaged_bids = Internal_Mix_Engaged(Engaged_bids)

FREE_BIDS = {1: Free_bids}
ENGAGED_BIDS = {1: Engaged_bids}


PRIVATE STABLE MATCHING ALGORITHM

In [ ]:
######### Computing a stable match ###########
for i in range(2, n+2):
    FREE_BIDS[i] = None
    ENGAGED_BIDS[i] = None

for k in range(1, n+1):
    while len(FREE_BIDS[k]) > 0:
        ind = random.randrange(0, len(FREE_BIDS[k]))
        bid = FREE_BIDS[k][ind]
        FREE_BIDS[k] = np.delete(FREE_BIDS[k], ind, 0)
        index = INDEX(bid[1], bid[4], private_key)
        E_j = bid[2][index]
        E_sji = bid[3][index]

        for i in range(len(ENGAGED_BIDS[k])):
            if EQ_TEST(E_j, ENGAGED_BIDS[k][i][1], private_key) == 1:
                conflict_bid = ENGAGED_BIDS[k][i]
                ENGAGED_BIDS[k] = np.delete(ENGAGED_BIDS[k], i, 0)
                break

        new_engaged_bid = np.array([np.array(bid), E_j, E_sji])

        temp_Engaged_bids = np.array([new_engaged_bid, conflict_bid])
        result = COMPARE(
            temp_Engaged_bids[0][2], temp_Engaged_bids[1][2], private_key, 2*n)
      
        if not result:
            ENGAGED_BIDS[k] = np.append(
                ENGAGED_BIDS[k], [temp_Engaged_bids[0]], axis=0)

            w1 = temp_Engaged_bids[1][0]
            w1[4] = add_1(w1[4])

            if FREE_BIDS[k+1] is None:
                FREE_BIDS[k+1] = np.array([w1])
            else:
                FREE_BIDS[k+1] = np.append(FREE_BIDS[k+1], [w1], axis=0)

        else:

            ENGAGED_BIDS[k] = np.append(ENGAGED_BIDS[k], [temp_Engaged_bids[1]], axis=0)
            w1 = temp_Engaged_bids[0][0]
            w1[4] = add_1(w1[4])
            if FREE_BIDS[k+1] is None:
                FREE_BIDS[k+1] = np.array([w1])

            else:
                FREE_BIDS[k+1] = np.append(FREE_BIDS[k+1], [w1], axis=0)

        ENGAGED_BIDS[k] = External_Mix_Engaged(ENGAGED_BIDS[k])
        ENGAGED_BIDS[k] = Internal_Mix_Engaged(ENGAGED_BIDS[k])
        FREE_BIDS[k] = Internal_Mix_Free(FREE_BIDS[k])
        FREE_BIDS[k+1] = Internal_Mix_Free(FREE_BIDS[k+1])

    ENGAGED_BIDS[k+1] = ENGAGED_BIDS[k]

    FREE_BIDS[k+1] = External_Mix_Free(FREE_BIDS[k+1])
    ENGAGED_BIDS[k+1] = External_Mix_Engaged(ENGAGED_BIDS[k+1])
    FREE_BIDS[k+1] = Internal_Mix_Free(FREE_BIDS[k+1])
    ENGAGED_BIDS[k+1] = Internal_Mix_Engaged(ENGAGED_BIDS[k+1])


DECRYPTION

In [ ]:
pairs = []
for bid in ENGAGED_BIDS[n+1]:
    pairs.append([bid[0][0], bid[1]])


for pair in pairs:
    pair = [re_encrypt(pair[0]), re_encrypt(pair[1])]
sigma = np.random.permutation(len(pairs))
pairs = [pairs[i] for i in sigma]

#########Decryption ################
for i in range(len(pairs)):
    pairs[i] = [private_key.decrypt(
        pairs[i][0]), private_key.decrypt(pairs[i][1])]

print(pairs)